In [ ]:
import sys

# Select path to current folder and split by \\
main_path = sys.path[0].split("\\")

# Asssign path to parent folder
# path_to_parent allows access to any folder from within parent folder, no matter the location of this file within the parent folder
# i.e.: Don't need to specify "../" x amount of times
path_to_parent = []
for element in main_path:
    path_to_parent.append(element)
    if "Fake_Users_Movies_Classifier" == element:
        break

path_to_parent = "\\".join(path_to_parent)

# Add path to feature generation folder
sys.path.append(path_to_parent+"\\feature_generation")

In [ ]:
# Import feature generator for week 2
from feature_gen_wk2 import feature_gen
import pandas as pd

# Create string path to labelled data
path_to_file1 = path_to_parent + "/data/labelled_data/first_batch_with_labels_likes.npz"
path_to_file2 = path_to_parent + "/data/labelled_data/second_batch_with_labels_likes.npz"

# Generate features from file
df1 = feature_gen().retrieveAndGenerate(path_to_file1)
df2 = feature_gen().retrieveAndGenerate(path_to_file2)

df_final = pd.concat([df1,df2]).reset_index(drop=True)

In [ ]:
from sklearn.svm import OneClassSVM
from sklearn.cluster import DBSCAN, KMeans
from sklearn.preprocessing import StandardScaler

# Retrieve labels and assign to y
y = df_final['label']
# Remove labels and users from dataset and assign to x
X = df_final.drop(['user','label'],axis=1)

# Apply scaler on X
X = StandardScaler().fit_transform(X)

# Define unsupervised classifiers and fit to X
ocSVM = OneClassSVM(gamma='auto', kernel='rbf').fit(X)
ocSVMLIN = OneClassSVM(gamma='auto', kernel='linear').fit(X)
kmeans = KMeans(n_clusters=2, n_init=10).fit(X)
dbs = DBSCAN(eps=4, min_samples=4)

# Predict labels using each classifier for X
ocSVMpred = ocSVM.predict(X)
ocSVMLINpred = ocSVMLIN.predict(X)
kmeanspred = kmeans.predict(X)
dbspred = dbs.fit_predict(X)

In [ ]:
# Change labels of predictions to correct format
ocSVMpred[ocSVMpred == -1] = 0
ocSVMLINpred[ocSVMLINpred == -1] = 0
dbspred[dbspred == 0] = 1
dbspred[dbspred == -1] = 0

In [ ]:
# Load class with getScores function for calulating predictions scores
from feature_selection import selectors

# Print scores for each classifier predictionss
print(f"One Class SVM: {selectors.getScores(true=y, pred=ocSVMpred)}")
print(f"One Class SVM Linear: {selectors.getScores(true=y, pred=ocSVMLINpred)}")
print(f"KMeans: {selectors.getScores(true=y, pred=kmeanspred)}")
print(f"DBSCAN: {selectors.getScores(true=y, pred=dbspred)}")